# 기본환경 설정

In [ ]:
import unsloth

In [ ]:
from google.colab import userdata
HF_KEY = userdata.get("HF_KEY")

In [ ]:
import huggingface_hub
huggingface_hub.login(HF_KEY)

In [ ]:
import langgraph_tool_smithy as smith

# 서비스 구조 생성

In [ ]:
from langgraph.graph import StateGraph, END
from IPython.display import Image, display

## Graph Node 생성

In [ ]:
def buildNode():
    graph = StateGraph(smith.StateToolQA)
    graph.add_node("tool_select", smith.node_tool_select)
    graph.add_node("tool_call", smith.node_tool_call)
    graph.add_node("question", smith.node_question)

    return graph

## Router 구성

In [ ]:
def after_tool_select(state: smith.StateToolQA) -> str:
    selection = state.get("selection", {}) or {}
    tool = selection.get("tool", "none")
    return "tool_call" if tool in smith.TOOL_FUNCS else "question"

## Graph Edge 구성

In [ ]:
def buildApp():
    # graph node 생성
    graph = buildNode()

    # graph 연결
    graph.set_entry_point("tool_select")
    graph.add_conditional_edges("tool_select", after_tool_select, {
        "tool_call": "tool_call",
        "question": "question",
    })
    graph.add_edge("tool_call", "question")
    graph.add_edge("question", END)

    return graph.compile()

In [ ]:
app = buildApp()

In [ ]:
display(Image(app.get_graph().draw_png(), width=200))

# 실행 Helper

In [ ]:
def question_answer(user_input: str):
    init: GraphState = {
        "input": user_input,
        "tool_schema": smith.TOOLS,
        "tool_names": ", ".join(smith.TOOLS.keys()),
    }
    out = app.invoke(init)
    return out.get("answer")

# 실행

In [ ]:
answer = question_answer("서울 날씨 어때?")
print(f"{'-'*50}\n{answer}")

In [ ]:
answer = question_answer("2와 3을 더한 결과는?")
print(f"{'-'*50}\n{answer}")

In [ ]:
answer = question_answer("좋은 회의 아이스브레이커 알려줘")
print(f"{'-'*50}\n{answer}")